<a href="https://colab.research.google.com/github/kenclements/praxis/blob/main/practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Tue Jan 24 02:46:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 117.2 MB/s eta 0:00:00


In [8]:
import pandas as pd

import tensorflow as tf

import keras

import torch

from transformers import BertTokenizer, BertModel

from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sb

In [11]:
#email = pd.read_csv('../../../../../../data/r4.2/email.csv')
file = pd.read_csv('/content/drive/MyDrive/Praxis/data/file.csv')
#http = pd.read_csv('../../../../../../data/r4.2/http.csv')
#logon = pd.read_csv('../../../../../../data/r4.2/logon.csv')
#device = pd.read_csv('../../../../../../data/r4.2/device.csv')
#tweets = pd.read_csv('/content/drive/MyDrive/Praxis/code/tweets.csv')
#enron = pd.read_csv('../../../../../../data/praxis/ENRON/emails_enron.csv')


In [16]:
#files = [file,email,http,logon,device]

In [15]:
#for x in files:
#    print(x.shape)
file.shape

(445581, 6)

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
named_params = list(model.named_parameters())

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer.encode_plus('Ken is the best!')

{'input_ids': [101, 6358, 2003, 1996, 2190, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
response = model(torch.tensor(tokenizer.encode('Ken is the best')).unsqueeze(0))

In [ ]:
response

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1575,  0.1448, -0.0683,  ..., -0.2043,  0.2934,  0.2390],
         [ 0.0336, -0.8281, -0.0863,  ..., -0.4555,  0.7815,  0.0969],
         [-0.3097, -0.4105,  0.0555,  ...,  0.0817,  0.9685,  0.6817],
         [-0.6090, -0.9053, -0.0094,  ..., -0.2928,  1.2827, -0.1907],
         [-0.8572, -0.4660, -0.3183,  ...,  0.8195,  0.9794, -0.5100],
         [ 0.8410,  0.0964, -0.0524,  ...,  0.0601, -0.5761, -0.3895]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8167, -0.2347,  0.4112,  0.5842, -0.2368, -0.0547,  0.8562,  0.1426,
          0.2936, -0.9997,  0.1450,  0.0200,  0.9754, -0.2654,  0.9061, -0.5125,
         -0.0333, -0.4753,  0.2095, -0.6647,  0.5949,  0.9258,  0.6194,  0.2113,
          0.3049,  0.1423, -0.4651,  0.8916,  0.9368,  0.6442, -0.5724,  0.1174,
         -0.9778, -0.1278,  0.3968, -0.9710,  0.1264, -0.6902,  0.0774,  0.0675,
         -0.8322,  0.1926,  0.9963, -0.6426, -

In [ ]:
response.pooler_output.shape

torch.Size([1, 768])

In [ ]:
model.pooler

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [ ]:
cls_embedding = response.last_hidden_state[:, 0, :].unsqueeze(0)

In [ ]:
cls_embedding.shape

torch.Size([1, 1, 768])

In [ ]:
model.pooler(cls_embedding).shape

torch.Size([1, 768])

In [ ]:
(model.pooler(cls_embedding) == response.pooler_output).all()

tensor(True)

In [ ]:
'sinan' in tokenizer.vocab

False

In [ ]:
i=0

In [ ]:
while i < 10:
    encoded_file_content = tokenizer.encode(file['content'][0])

In [ ]:
file.size

2673486

In [ ]:
encoded_file_content[7]

[101,
 1040,
 2692,
 1011,
 12935,
 1011,
 2340,
 1011,
 1041,
 2692,
 1011,
 17350,
 1011,
 29491,
 1011,
 20720,
 1011,
 1041,
 2487,
 2350,
 2150,
 2116,
 2230,
 4265,
 3225,
 15175,
 4789,
 28740,
 2015,
 8198,
 18778,
 21662,
 2717,
 2125,
 4810,
 2070,
 2021,
 2456,
 2633,
 2757,
 5883,
 2570,
 1039,
 4198,
 5797,
 2952,
 9037,
 5550,
 16232,
 2207,
 14712,
 2207,
 2043,
 4653,
 2853,
 11397,
 4078,
 8586,
 9250,
 8494,
 3149,
 2332,
 2264,
 6368,
 2590,
 2612,
 2024,
 4746,
 2372,
 2789,
 6926,
 102]